In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests

# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved/created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Cape Town,ZA,-33.93,18.42,60.01,58,75,8.05,broken clouds,0.0,0.0
1,1,Carnarvon,AU,-24.87,113.63,59.00,72,0,6.93,clear sky,0.0,0.0
2,2,Grand Gaube,MU,-20.01,57.66,73.00,66,92,11.99,overcast clouds,0.0,0.0
3,3,Ushuaia,AR,-54.80,-68.30,30.20,80,40,1.99,scattered clouds,0.0,0.0
4,4,Saskylakh,RU,71.92,114.08,66.49,77,80,11.83,light rain,0.6,0.0


In [3]:
# Get the data types.
city_data_df.dtypes

City_ID                  int64
City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
Rain (inches)          float64
Snow (inches)          float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
city_rain = city_data_df["Rain (inches)"]
city_snow = city_data_df["Snow (inches)"]

In [6]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

# Ask the customer to add preference for rain and snow.
rain = input("Do you want it to be raining? (yes/no) ")
if rain == "no":
    rain == 0
else:
    rain > 0

snow = input("Do you want it to be snowing? (yes/no) ")
if snow == "no":
    city_snow == 0
else:
    city_snow > 0    

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [7]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                       (city_data_df["Rain (inches)"] == city_rain) & \
                                       (city_data_df["Snow (inches)"] == city_snow)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
14,14,Pangoa,PE,-12.12,-73.00,75.67,60,1,2.33,clear sky,0.0,0.0
16,16,Balotra,IN,25.83,72.23,94.35,58,99,22.88,overcast clouds,0.0,0.0
18,18,Samarai,PG,-10.62,150.67,77.72,88,99,15.79,moderate rain,1.3,0.0
19,19,Belmonte,BR,-15.86,-38.88,78.80,65,20,5.82,few clouds,0.0,0.0
25,25,Katsuura,JP,35.13,140.30,80.01,94,75,13.87,broken clouds,0.0,0.0
26,26,Abeche,TD,13.83,20.83,84.34,49,3,11.70,clear sky,0.0,0.0
29,29,Port Blair,IN,11.67,92.75,83.01,81,100,9.84,overcast clouds,0.0,0.0
30,30,Albany,US,42.60,-73.97,81.00,82,98,8.55,overcast clouds,0.0,0.0
32,32,Hithadhoo,MV,-0.60,73.08,84.54,75,10,10.74,clear sky,0.0,0.0
39,39,Hirado,JP,33.36,129.55,75.20,100,75,5.82,heavy intensity shower rain,0.0,0.0


In [8]:
# Determine if any null values.
preferred_cities_df.count()

City_ID                244
City                   244
Country                244
Lat                    244
Lng                    244
Max Temp               244
Humidity               244
Cloudiness             244
Wind Speed             244
Current Description    244
Rain (inches)          244
Snow (inches)          244
dtype: int64

In [9]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
14,Pangoa,PE,75.67,clear sky,-12.12,-73.00,
16,Balotra,IN,94.35,overcast clouds,25.83,72.23,
18,Samarai,PG,77.72,moderate rain,-10.62,150.67,
19,Belmonte,BR,78.80,few clouds,-15.86,-38.88,
25,Katsuura,JP,80.01,broken clouds,35.13,140.30,
26,Abeche,TD,84.34,clear sky,13.83,20.83,
29,Port Blair,IN,83.01,overcast clouds,11.67,92.75,
30,Albany,US,81.00,overcast clouds,42.60,-73.97,
32,Hithadhoo,MV,84.54,clear sky,-0.60,73.08,
39,Hirado,JP,75.20,heavy intensity shower rain,33.36,129.55,


In [10]:
# Set parameters to search for a hotel.
params = {"radius": 5000,
          "type": "lodging",
          "key": g_key}

In [11]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError, NameError):
        print("Hotel not found... skipping.")
        
hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
14,Pangoa,PE,75.67,clear sky,-12.12,-73.00,
16,Balotra,IN,94.35,overcast clouds,25.83,72.23,Hotel City Square
18,Samarai,PG,77.72,moderate rain,-10.62,150.67,Nuli Sapi
19,Belmonte,BR,78.80,few clouds,-15.86,-38.88,Pousada Monte Carmelo O Pão
25,Katsuura,JP,80.01,broken clouds,35.13,140.30,Katsuura Hotel Mikazuki
26,Abeche,TD,84.34,clear sky,13.83,20.83,Assaleh Hamdan Haroune
29,Port Blair,IN,83.01,overcast clouds,11.67,92.75,Hotel Sentinel
30,Albany,US,81.00,overcast clouds,42.60,-73.97,
32,Hithadhoo,MV,84.54,clear sky,-0.60,73.08,Scoop Guest House
39,Hirado,JP,75.20,heavy intensity shower rain,33.36,129.55,Samson Hotel


In [12]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_vacation.csv"

# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# Add info box template information.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [14]:
# Add a marker layer for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))